In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/code/bettycxh06/rri-amplitudes/input
!kaggle datasets download -d bettycxh06/apnea-ecg-database

!kaggle kernels pull bettycxh06/rri-amplitudes


 97% 305M/315M [00:02<00:00, 184MB/s]
100% 315M/315M [00:02<00:00, 144MB/s]
Source code downloaded to /content/rri-amplitudes.ipynb


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
os.listdir(os.getcwd())

['.config',
 'apnea-ecg-database.zip',
 'rri-amplitudes.ipynb',
 'kaggle.json',
 'sample_data']

In [ ]:
!unzip apnea-ecg-database.zip

Archive:  apnea-ecg-database.zip
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/ANNOTATORS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/RECORDS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/SHA256SUMS.txt  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.dat  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01r.apn  

In [ ]:
!pip install biosppy
!pip install wfdb

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import csv
import wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
#base_dir = "../input/apnea-ecg-database/apnea-ecg-database-1.0.0"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"



fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



    answers = {}
    with open(os.path.join("event-2-answers"), "r") as f:
    #    with open(os.path.join(base_dir, "event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:

        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
c09:   6%|▌         | 28/468 [03:48<59:44,  8.15s/it]

a01:   7%|▋         | 32/489 [04:03<58:01,  7.62s/it]



c07:   7%|▋         | 28/429 [03:48<54:36,  8.17s/it]










a09:   6%|▋         | 31/495 [04:00<59:55,  7.75s/it]  

b02:   6%|▌         | 31/517 [03:53<1:00:59,  7.53s/it]

a12:   5%|▍         | 26/577 [03:59<1:24:26,  9.20s/it]


a17:   5%|▍         | 24/485 [03:56<1:15:48,  9.87s/it]

c10:   8%|▊         | 34/431 [03:47<44:18,  6.70s/it]







In [ ]:
!pip install keras
!pip install tensorflow
#
#IMPORT LIBRARIES
import pickle
import numpy as np
import tensorflow
import os
from keras.callbacks import LearningRateScheduler,EarlyStopping
from keras.layers import Dense,Flatten,MaxPooling2D,Conv2D,BatchNormalization
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Model, load_model, save_model
from keras.layers import Input
import pandas as pd

#base_dir = "dataset"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"

#------------------------------------------------------------------------------
# DEEP LEARNING MODELS NEED TO VECTORS OR MATRIX WITH SAME SIZE
# R-R INTERVALS DONT HAVE SAME SIZE, SO WE NEED TO INTERPOLATE VECTORS TO GET VECTORS WITH SAME SIZE.
# BASED ON OUR EXPERIENCE INTERPOLATION IN 3 HZ BETTER AND ACCURATE.
ir = 3 # INTERPOLATION RATE(3HZ)
time_range= 60 # 60-s INTERVALS OF ECG SIGNALS
weight=1e-3 #  WEIGHT L2 FOR REGULARIZATION(AVODING OVERFITTING PARAMETER)
#-----------------------------
# NORMALIZATION:
# DEEP LEARNING AND EVEN NEURAL NETWORKS INPUT SHOULD BE NORMALIZED:
# MIN-MAX METHOD APPLIED FOR SCALING:(Array-min(Array))/(max(Array)-min(Array))
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
#-----------------------------
# FIRSTLY WE PRE-PROCESSED OUR DATA IN "apnea-ecg.pkl" FILE
# IN PRE-PROCESSING SECTION WE EXTRACT R-R INTERVALS AND R-PEAK AMPLITUDES
# IN THIS PART WE LOAD THIS DATA AND INTERPOLATE AND CONCATE FOR FEEDING TO NETWORKS
def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)
#-----------------
    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]

    for i in range(len(o_train)):
        (rri_tm, rri_signal), (amp_tm, amp_signal) = o_train[i]
      # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        amp_interp_signal = splev(tm, splrep(amp_tm, scaler(amp_signal), k=3), ext=1)
        x_train.append([rri_interp_signal, amp_interp_signal])
    x_train = np.array(x_train, dtype="float32")

    x_train = np.expand_dims(x_train,1)
    x_train=np.array(x_train, dtype="float32").transpose((0,3,1,2)) # convert to numpy format

    #return x_train_final, y_train
    return x_train, y_train

In [ ]:
!pip install Hyperactive
!pip install mealpy
!pip install tensorflow
!pip install optimizers
!pip install keras

#from keras.layers.rnn import GRU
from keras.layers import Reshape, Lambda
from keras.layers import GRU
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer
from hyperactive.optimizers import RandomSearchOptimizer
from mealpy.swarm_based.GWO import OriginalGWO

from tensorflow.keras import optimizers
from keras.optimizers import Adam
import threading
import concurrent.futures
from threading import Thread
import time
from tabnanny import verbose
from mealpy.swarm_based.HBA import OriginalHBA

from tensorflow.python.keras import optimizers
#from keras.optimizers import RandomSearchOptimizer

import warnings
warnings.filterwarnings('ignore')


In [ ]:
#print(x_train.shape, y_train.shape)
#x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
#x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
#x_train= x_train.transpose((0,2,1))
#print(x_train.shape, y_train.shape)
##(16709, 900, 1, 2) (16709, 2)
##(16709, 2, 900) (16709, 2)


In [ ]:
# BHA
###########################
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,epochs = position[0],position[1],position[2],position[3],position[4],position[5]
    n1,n2,n3,n4,epochs=int(n1),int(n2),int(n3),int(n4),int(epochs)

    print(learning_rate,n1,n2,n3,n4,epochs)

#def create_model(weight=1e-3):
    model= Sequential()
    model.add(GRU(n1, return_sequences=True, input_shape=(2,900)))     #(2,900)))
    model.add(GRU(n2, return_sequences=True))
    model.add(GRU(n3, return_sequences=True))

    model.add(Flatten())

    model.add(Dense(n4, activation="relu"))
    model.add(Dense(2, activation="softmax"))

    return learning_rate,n1,n2,n3,n4,epochs


    optimizer =keras.optimizers.Adam(lr=learning_rate)
    print("Learning rate: ", learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

    history=model.fit(X1, Y1, batch_size=128, epochs=int(epochs), validation_data=(x_val, y_val),
                       callbacks=[callback1,lr_scheduler])


    #history = model.fit(x_train,y_train, epochs=int(epochs),batch_size=32, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1]
    #loss, accuracy = model.evaluate(x_test, y_test)

    #return model
#------------------------------------------------------------------------------
# Define learning rate schedule for preventing overfitting in deep learning methods:
def lr_schedule(epochs, learning_rate):
   if epochs > 70 and \
           (epochs - 1) % 10 == 0:
        learning_rate *= 0.1
   print("Learning rate: ", learning_rate)
   return learning_rate

#------------------------------------------------------------------------------

        # we used k-fold cross-validation for more reliable experiments:
   kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)
   cvscores = []
   ACC=[]
   SN=[]
   SP=[]
   F2=[]
   print("train num:", len(y_train))
   print("test num:", len(y_test))


    # separate train& test and then compile model
  # for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

     # Compile and evaluate model:
if __name__ == "__main__":
    # loading Data:
    x_train, y_train = load_data()
    # we have labels(Y) in a binary way 0 for normal and 1 for apnea patients
    # we want to classify data into 2-class so we changed y in a categorical way:
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    # we used k-fold cross-validation for more reliable experiments:
    kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)


    # separate train& test and then compile model
    ##for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

    # define callback for early stopping:
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callback1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
    x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
    x_train= x_train.transpose((0,2,1))
    #10% of Data used for validation:
    X1,x_val,Y1,y_val=train_test_split(x_train,y_train,test_size=0.10)

##################################################################
#pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
##bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])
#v = func(bp);
##################################################################
problem_dict1 = {
    "fit_func": fitness_function,
    'lb':[0.001,80,35,18,5,30], # Lower bound of our parameters
    'ub':[0.1,110,55,32,14,200], # upper bound of our parameters
    "minmax": "min",
    "obj_weights": [0.4,1,1,1,1,1]               # Define it or default value will be [1, 1, 1]   [0.4, 0.1, 0.5,1,1,1,1,1]
    #"mode":"thread"
    #"amend_position": amend_position

                }

rng = np.random.default_rng()
epoch = 100
pop_size =50

#def thread_function(k):

  #run=k+1
  #print("Run Number",run)
  #Optimizer = MHoneyBadgerAlgorithm(search_config, n_iter=100, n_part=10, metric='accuracy', cv=10, h_beta=6.0, h_c=2.0,run=run)
model = OriginalGWO(epoch, pop_size)
  #t1 = time.time()

#best_position, best_fitness = model.solve(problem_dict1)
best_position, best_fitness= model.solve(problem_dict1)
#best_position, best_fitness= model.solve(mode="thread")
v = fitness_function(best_position);
#print(f"Solution: {best_position}, Fitness: {best_fitness}")
#return v
print(f"Solution: {best_position}, Fitness: {v}")

0.06916877673247603 82 50 19 6 50


INFO:mealpy.swarm_based.GWO.OriginalGWO:Solving 6-objective optimization problem with weights: [0.4 1.  1.  1.  1.  1. ].


0.09137294255640604 99 45 26 13 40
0.07805835843005755 102 38 31 10 37
0.009356144833099034 83 35 19 7 57
0.049953935067606633 105 39 19 5 148
0.09206098826082917 84 44 25 8 163
0.09694619616767242 88 46 30 5 179
0.03894389469187335 97 50 19 6 128
0.07253586543303972 109 54 26 10 111
0.018927700476255287 85 46 31 6 100
0.009432382108517764 87 39 18 5 123
0.02657579431849714 97 40 31 8 45
0.09315342267878013 91 54 22 6 88
0.03580027555683111 98 46 21 7 189
0.09677673053339433 97 45 21 7 130
0.05420488907335771 82 54 20 9 151
0.04561772329573682 99 53 19 11 97
0.031783131555936976 80 52 30 10 172
0.057703216528902715 94 47 24 13 90
0.02064375126399072 88 50 20 6 110
0.06373957965714848 89 53 21 10 78
0.018905344308422316 94 53 19 12 66
0.011463731972481132 106 36 27 13 174
0.0882219271006753 98 37 28 9 177
0.06459306810860488 81 45 24 8 31
0.0937204320189821 98 51 19 5 125
0.046528203448295724 90 40 29 8 143
0.03824670099027602 95 41 24 10 79
0.022429964259290353 81 40 30 5 152
0.0868819

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 1, Current best: 171.01565317087187, Global best: 171.01565317087187, Runtime: 43.10439 seconds


0.03137568832730328 80 55 18 10 41
0.04886451971898085 104 42 18 6 35
0.01955019074978776 103 52 24 5 30
0.04073144008206762 87 50 26 7 30
0.05065442054997188 85 35 18 9 31
0.014452269705311847 80 35 18 11 142
0.04615066946228711 82 35 25 10 30
0.001 80 50 22 5 50
0.1 80 55 18 11 30
0.08870124820485621 110 44 20 6 30
0.026330501303427994 80 35 18 6 45
0.03408967408228949 104 35 32 5 74
0.04963600389547327 80 50 18 5 36
0.06279076320242756 80 35 27 5 44
0.037076174699497276 93 40 32 6 30
0.06483621684966441 80 35 18 5 40
0.001 90 37 21 6 58
0.032370163984026284 80 40 18 7 30
0.04260010177348233 110 35 18 5 30
0.03669993558836182 81 39 18 9 32
0.02940035309704794 80 55 21 5 30
0.07777247456538916 83 35 19 11 40
0.011119095831125263 80 35 18 5 30
0.05186237745848169 97 35 19 8 30
0.045160131398540276 88 35 27 7 30
0.020607843850272707 85 35 26 5 30
0.05478686663586856 80 48 18 8 33
0.03403508704343785 85 35 22 8 42
0.030471319114490063 80 37 18 8 30
0.03608126133405722 80 45 19 6 64
0.035

...

0.0010050873419496923 80 35 18 5 30
0.001000187186523317 80 35 18 5 30
0.0010013777356886933 80 35 18 5 30
0.0010025366928220838 80 35 18 5 30
0.0010093036184608498 80 35 18 5 30
0.001001523796078911 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.0010008108357655537 80 35 18 5 30
0.001 80 35 18 5 30
0.0010029766974895507 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001001834871374369 80 35 18 5 30
0.001000932199464893 80 35 18 5 30
0.001004117971677866 80 35 18 5 30
0.001001412934926682 80 35 18 5 30
0.0010049184339999625 80 35 18 5 30
0.001002568229920777 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001002292880270357 80 35 18 5 30
0.0010029255365974722 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.0010072231002304464 80 35 18 5 30
0.001001794914914202 80 35 18 5 30
0.001 80 35 18 5 30
0.0010006982974199977 80 35 18 5 30
0.001 80 35 18 5 30
0.001003074378627432 80 3

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 99, Current best: 168.0004, Global best: 168.0004, Runtime: 42.20012 seconds


0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30
0.001 80 35 18 5 30


INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 100, Current best: 168.0004, Global best: 168.0004, Runtime: 41.83111 seconds


0.001 80 35 18 5 30
Solution: [1.00000000e-03 8.00000000e+01 3.50000000e+01 1.88550598e+01
 5.40415168e+00 3.00000000e+01], Fitness: (0.001, 80, 35, 18, 5, 30)
